In [1]:
import numpy as np
import tensorflow as tf

In [2]:
class MLP:
    def __init__(self,data_path, model_path, hidden_node_num):
        #Define instance variable
        self.__data_path = data_path
        self.__model_path = model_path
        self.__hidden_node_num = hidden_node_num
        self.__hidden_layer_num = len(self.__hidden_node_num)
        
        self.__data = None
        self.__data_len = None
        
        self.__y = None
        self.__x = None
        
        self.__w = []
        self.__b = []
        self.__layer = []
        
        self.__y_out = None
        self.__train = None
        self.__sess = None
        self.__saver = None
        
        self.__result = []
        
        self.__loadData()
        self.__init_sess()
        
    def __loadData(self):
        tf.reset_default_graph()
                
        xy = np.genfromtxt(self.__data_path, dtype='float32')
        data_num = xy[:,1:].shape[0]
        self.__data =  xy[:,1:]

        self.__data = self.__data.transpose()
        self.__data_len = len(self.__data)
        
        self.__x = tf.placeholder(dtype=tf.float32)
        
        # first layer
        self.__w.append(tf.Variable(tf.random_normal([self.__hidden_node_num[0], self.__data_len]), name="w0"))
        self.__b.append(tf.Variable(tf.random_normal([self.__hidden_node_num[0],1]), name="b0"))
       
        # add hidden layers (variable number)
        for i in range(1,self.__hidden_layer_num):
            wName = "w" + str(i)
            bName = "b" + str(i)
            self.__w.append(tf.Variable(tf.random_normal([self.__hidden_node_num[i], self.__hidden_node_num[i-1]]), name=wName))
            self.__b.append(tf.Variable(tf.random_normal([self.__hidden_node_num[i],1]), name=bName))
        
        # add final layer
        wName = "w" + str(self.__hidden_layer_num)
        bName = "b" + str(self.__hidden_layer_num)
        self.__w.append(tf.Variable(tf.random_normal([1, self.__hidden_node_num[-1]]), name=wName))
        self.__b.append(tf.Variable(tf.random_normal([1],1), name=bName))
        
        # define model
        self.__layer.append(tf.nn.sigmoid(tf.matmul( self.__w[0],self.__x) + self.__b[0]))
        for i in range(1,self.__hidden_layer_num):
            self.__layer.append(tf.nn.sigmoid(tf.matmul( self.__w[i],self.__layer[i-1]) + self.__b[i]))
        self.__y_out = tf.matmul(self.__w[-1],self.__layer[-1]) + self.__b[-1]
        
    def __init_sess(self):
        init = tf.global_variables_initializer()
        self.__sess = tf.Session()
        self.__sess.run(init)
        self.__saver = tf.train.Saver()
        self.__saver.restore(self.__sess, self.__model_path)
        self.__result.append(self.__sess.run(self.__y_out,feed_dict={self.__x: self.__data}))
        
    def getResult(self):
        return self.__result[0][0]

In [3]:
hidden_node_num = [10,10,10]

alpha_num = 5
start_alpha = 0
end_alpha = 4


In [4]:
dataPath = "test.txt"
savePath = "result.txt"
model_path = "model/0222_1512/model_"
result = []
for i in range(start_alpha,end_alpha+1):
    model_paths = model_path + str(i)
    myMLP = MLP(dataPath, model_paths,hidden_node_num)
    result.append(myMLP.getResult())

INFO:tensorflow:Restoring parameters from model/0222_1512/model_0
INFO:tensorflow:Restoring parameters from model/0222_1512/model_1
INFO:tensorflow:Restoring parameters from model/0222_1512/model_2
INFO:tensorflow:Restoring parameters from model/0222_1512/model_3
INFO:tensorflow:Restoring parameters from model/0222_1512/model_4


In [9]:
saveOpen = open(savePath,'w')
result_len = len(result[0])

print(type(result[0]))
for i in range(result_len):
    myStr = ""
    for j in range(alpha_num):
        myStr += result[i][j] + "\t"
    myStr += "\n"
saveOpen.close()
    

<class 'numpy.ndarray'>


TypeError: ufunc 'add' did not contain a loop with signature matching types dtype('<U32') dtype('<U32') dtype('<U32')